In [2]:
!pip install sounddevice

     ------------------------------------ 199.7/199.7 kB 865.2 kB/s eta 0:00:00


In [1]:
import tensorflow_hub as hub
import pandas as pd
from datetime import datetime
import numpy as np
import requests
import zipfile, tarfile
import librosa
import tensorflow as tf
import os, gc, io, time, traceback
from scipy.signal import lfilter, lfilter_zi, firwin
import pywt
import sounddevice as sd
import queue
import threading
from ipywidgets import Button, Layout, ButtonStyle
from IPython.display import display

In [12]:
# Скачиваем модель Yamnet (делаем один раз при первом запуске скрипта)

# URL для скачивания модели
model_url = 'https://storage.googleapis.com/tfhub-modules/google/yamnet/1.tar.gz'
# Путь для сохранения модели
model_path = './yamnet_model'

# Создание каталога, если он не существует
if not os.path.exists(model_path):
    os.makedirs(model_path)
    
# Скачивание и распаковка модели
response = requests.get(model_url)
tar_path = os.path.join(model_path, 'yamnet.tar.gz')
with open(tar_path, 'wb') as file:
    file.write(response.content)

# Распаковка вложенного TAR.GZ архива
with tarfile.open(tar_path, 'r:gz') as tar_ref:
    tar_ref.extractall(model_path)

# Удаление архива после распаковки
os.remove(tar_path)

In [2]:
# загружаем файл классов Yamnet
class_map_path = 'yamnet_class_map_translated.csv'
class_map = pd.read_csv(class_map_path, sep=';', encoding='Windows-1251')

# Загрузка модели
model_path = './yamnet_model'
yamnet_model = hub.load(model_path)

In [4]:
class_map

,index,mid,display_name,display_name_ru
0,0,/m/09x0r,Speech,Речь
1,1,/m/0ytgt,"Child speech, kid speaking","Детская речь, говорящий ребенок"
2,2,/m/01h8n0,Conversation,Разговор
3,3,/m/02qldy,"Narration, monologue","Повествование, монолог"
4,4,/m/0261r1,Babbling,Бормочущий
...,...,...,...,...
516,516,/m/07p_0gm,Throbbing,Пульсирующий
517,517,/m/01jwx6,Vibration,Вибрация
518,518,/m/07c52,Television,Телевидение
519,519,/m/06bz3,Radio,Радио


In [18]:
# Параметры

sample_rate=44100 # частота дискретизации
preemphasis_coef=None #0.25 # Коэффициент преэмфазного шумоподавления
adaptive_filtered=False # Применять ли адаптивную фильтрацию сигнала
wavelet_filtered=False # Применять ли вейвлет-шумоподавление сигнала
n=512 # количество коэффициентов адаптивного фильтра
noise_level = 0.5 # порог адаптивного шумоподавления
probability = 0.65 # порога уверенности классификации моделью YAMNet
time_threshold=15.0 # интервал ожидания повторения такого же события (в секундах)
duration = 5 # размер буферного окна захвата потока

processing = True  # Глобальная переменная для контроля состояния обработки

# Функция адаптивного фильтра для уменьшения шума
def adaptive_filter(y, noise_level, n):
    b = firwin(n, cutoff=noise_level, window='hamming')
    z = lfilter_zi(b, 1)
    filtered_signal, _ = lfilter(b, 1, y, zi=z*y[0])
    return filtered_signal

# Функция шумоподавления на вейвлет-преобразованиях
def wavelet_denoise(data, wavelet='db8', level=3):
    coeffs = pywt.wavedec(data, wavelet, level=level)
    limit = np.std(coeffs[-level]) / 10
    thresholded_coeffs = [pywt.threshold(c, limit, mode='soft') if i == level else c for i, c in enumerate(coeffs)]
    return pywt.waverec(thresholded_coeffs, wavelet)

# Функция для предобработки и анализа потоковых аудиоданных
def preprocess_stream_audio(audio_data, model, sample_rate, preemphasis_coef, adaptive_filtered, wavelet_filtered):
    # Обработка аудиоданных, поступающих в виде потока
    y = np.array(audio_data).flatten()

    # Применяем преэмфазное шумоподавление для выделения высокочастотных компонентов аудиосигнала
    if preemphasis_coef is not None:
        y = librosa.effects.preemphasis(y, coef=preemphasis_coef)
        #y = librosa.util.normalize(y)

    # Применение адаптивного фильтра для очистки изменяющегося фонового шума
    if adaptive_filtered:
        y = adaptive_filter(y, noise_level, n)
        #y = librosa.util.normalize(y)

    # Применяем вейвлет-шумоподавление для финального удаления шума с сохранением деталей сигнала
    if wavelet_filtered:
        y = wavelet_denoise(y)
        #y = librosa.util.normalize(y)

    # Масштабирование (стандартизация) данных для корректной работы модели
    #y = (y - np.mean(y)) / np.std(y)
    #y = y / np.max(np.abs(y))
    
    # Анализ с помощью YAMNet
    scores, _, _ = model(y)

    return scores.numpy()

# Функция группировки череды одинаковых событий в одно общее событие
def group_events(descriptions, event_times, time_threshold, ongoing_event=None):
    if not event_times and not ongoing_event:
        return []

    grouped_events = []
    if ongoing_event:
        # Начинаем с продолжающегося события
        current_description, current_start, current_end = ongoing_event
    else:
        # Начинаем с первого события в текущем окне
        current_start, current_end = event_times[0]
        current_description = descriptions[0]

    for next_description, (next_start, next_end) in zip(descriptions, event_times):
        if current_description != next_description or (next_start - current_end) > time_threshold:
            # Завершаем текущее событие и начинаем новое
            grouped_events.append((current_description, current_start, current_end))
            current_description, current_start = next_description, next_start
        # Обновляем время окончания для текущего или продолжающегося события
        current_end = next_end
 
    # Не добавляем последнее событие, если оно может продолжаться
    # сохраняем его в ongoing_event для следующего блока
    ongoing_event = (current_description, current_start, current_end)
    return grouped_events, ongoing_event

# Функция для захвата и обработки потоковых аудиоданных
def stream_audio_processing(model, class_map, duration, sample_rate, probability, time_threshold):
    global processing
    start_time = time.time()  # Начальное время обработки
    formatted_start_time = datetime.fromtimestamp(start_time).strftime('%H:%M:%S')
    events_data = []  # Список для сохранения данных о событиях
    audio_data = queue.Queue()
    print(f'Начало обработки аудиопотока в {formatted_start_time}\n')
    
    # Callback для захвата аудио
    def callback(indata, frames, time, status):
        if status:
            print(status)
        audio_data.put(indata.copy())
    
    # Параметры YAMNet
    patch_hop_seconds = 0.48  # Из конфигурации YAMNet

    try:
        # Запуск захвата аудио
        with sd.InputStream(callback=callback, channels=1, samplerate=sample_rate, blocksize=int(sample_rate * duration)):
            cnt = 0
            processed_time = 0  # Общее время обработанных данных
            ongoing_event = None
            while processing:
                if not audio_data.empty():
                    # Получение и обработка аудиоданных
                    data = audio_data.get()
                    current_block_start_time = start_time + processed_time
                    processed_time += duration  # Увеличиваем общее время обработанных данных
                    
                    scores = preprocess_stream_audio(data, model, sample_rate, preemphasis_coef, adaptive_filtered, wavelet_filtered)
                    
                    # Получение индексов класса с максимальной вероятностью для каждого временного шага
                    class_ids = np.argmax(scores, axis=1)
                    class_descriptions = class_map.iloc[class_ids, 3].tolist()
                        
                    # Определение временных границ событий
                    events = []
                    event_start = None
                    time_step = duration / scores.shape[0]  # Вычисление временного шага в окне

                    for i in range(scores.shape[0]):
                        max_score = np.max(scores[i])
                        # Вычисляем абсолютное время для текущего шага в окне
                        current_time = current_block_start_time + i * time_step

                        if max_score > probability and event_start is None:
                            event_start = current_time
                        elif (max_score <= probability or i == scores.shape[0] - 1) and event_start is not None:
                            event_end = current_time
                            events.append((event_start, event_end))
                            event_start = None

                    # Преобразование индексов событий во временные метки
                    event_times = [(start, end) for start, end in events]

                    # Группировка событий
                    grouped_events, ongoing_event = group_events(class_descriptions, event_times, time_threshold, ongoing_event)

                    # Вывод и сохранение результатов
                    for (description, event_start, event_end) in grouped_events:
                        formatted_start = datetime.fromtimestamp(event_start).strftime('%H:%M:%S')
                        formatted_end = datetime.fromtimestamp(event_end).strftime('%H:%M:%S')
                        interval = np.round((event_end - event_start),2)
                        cnt += 1
                        print(f'{cnt}. {description}. Период действия события: {formatted_start} - {formatted_end} Длительность: {interval}')
                        events_data.append([cnt, description, formatted_start, formatted_end, interval])
                            
                #time.sleep(0.1)  # небольшая задержка, чтобы снизить загрузку ЦП
    except KeyboardInterrupt:
        print('Прервано пользователем')
    except Exception as e:
        print(f'Произошла ошибка: {e}')
        traceback.print_exc()

    finally:
        # Очистка очереди
        while not audio_data.empty():
            audio_data.get()        

        last_time = datetime.fromtimestamp(time.time()).strftime('%H:%M:%S')

        # Обработка последнего продолжающегося события
        if ongoing_event:
            description, event_start, event_end = ongoing_event
            formatted_start = datetime.fromtimestamp(event_start).strftime('%H:%M:%S')
            formatted_end = datetime.fromtimestamp(event_end).strftime('%H:%M:%S')
            interval = np.round((event_end - event_start),2)
            cnt += 1
            print(f'{cnt}. {description}. Период действия события: {formatted_start} - {formatted_end} Длительность: {interval}')
            events_data.append([cnt, description, formatted_start, formatted_end, interval])

        # Сохранение всех событий в CSV
        if len(events_data) > 0:
            events_df = pd.DataFrame(events_data, columns=['Номер', 'Событие', 'Начало', 'Окончание', 'Длительность'])
            directory = os.getcwd()
            csv_filename = f'{int(start_time)}_events.csv'
            csv_path = os.path.join(directory, csv_filename)
            events_df.to_csv(csv_path, index=False, encoding='utf-8-sig')
            print(f'\nПотоковая обработка остановлена в {last_time}. События сохранены в {csv_filename}')
        else:
            print(f'Потоковая обработка остановлена в {last_time}. За время наблюдения событий не произошло')

# Функция для запуска потока обработки аудио
def start_audio_processing_thread(model, class_map, duration, sample_rate, probability, time_threshold):
    global processing
    processing = True

    def audio_processing_thread():
        stream_audio_processing(model, class_map, duration, sample_rate, probability, time_threshold)

    processing_thread = threading.Thread(target=audio_processing_thread)
    processing_thread.start()
    
# Функция для остановки потоковой обработки            
def stop_processing(button):
    global processing
    processing = False
    button.description = 'Обработка остановлена. Для возобнавления перезапустите ячейку.'

# Создаем кнопку и событие для остановки обработки
stop_button = Button(
    description='Остановить обработку', 
    layout=Layout(width='500px', height='50px', border='solid'),
    style=ButtonStyle(button_color='lightblue', font_weight='bold')
)
stop_button.on_click(stop_processing)
display(stop_button)

# Запуск обработки потокового аудио
start_audio_processing_thread(model=yamnet_model, class_map=class_map, duration=duration, sample_rate=sample_rate, probability=probability, time_threshold=time_threshold)

Button(description='Остановить обработку', layout=Layout(border='solid', height='50px', width='500px'), style=…

Начало обработки аудиопотока в 22:06:55

1. Тишина. Период действия события: 22:06:55 - 22:07:30 Длительность: 34.82
2. Напевающий. Период действия события: 22:07:31 - 22:07:33 Длительность: 1.79
3. Тишина. Период действия события: 22:07:33 - 22:07:39 Длительность: 6.43
4. Белый шум. Период действия события: 22:07:41 - 22:07:42 Длительность: 0.71
5. Транспортное средство. Период действия события: 22:07:43 - 22:07:43 Длительность: 0.18
6. Белый шум. Период действия события: 22:07:44 - 22:07:45 Длительность: 0.54
7. Тишина. Период действия события: 22:07:45 - 22:07:50 Длительность: 4.82
8. Животное. Период действия события: 22:07:51 - 22:07:55 Длительность: 3.75
9. Речь. Период действия события: 22:07:56 - 22:08:00 Длительность: 4.64
